# Setup

## Libraries and Configurations

In [1]:
import pandas as pd
import os
import openai
from google import genai
from tqdm.auto import tqdm
import time
from bert_score import score
import os
from dotenv import load_dotenv

/Users/heshamnawaz/Desktop/Projects/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


https://www.kaggle.com/datasets/andrezaza/clapper-massive-rotten-tomatoes-movies-and-reviews

https://www.kaggle.com/datasets/stefanoleone992/rotten-tomatoes-movies-and-critic-reviews-dataset?resource=download



## Load Data

In [3]:
data_path = '/Users/heshamnawaz/Desktop/Projects/New Movie Review Summarization/data'

In [4]:
reviews = pd.read_csv(f'{data_path}/rotten_tomatoes_critic_reviews.csv')
movies = pd.read_csv(f'{data_path}/rotten_tomatoes_movies.csv')

In [5]:
movies.head(1)

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,...,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,...,20th Century Fox,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76


In [6]:
movies.describe()

,runtime,tomatometer_rating,tomatometer_count,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
count,17398.000000,17668.000000,17668.000000,17416.000000,1.741500e+04,17712.000000,17712.000000,17712.000000
mean,102.214048,60.884763,57.139801,60.554260,1.439401e+05,14.586326,36.374831,20.703139
std,18.702511,28.443348,68.370047,20.543369,1.763577e+06,15.146349,52.601038,30.248435
min,5.000000,0.000000,5.000000,0.000000,5.000000e+00,0.000000,0.000000,0.000000
25%,90.000000,38.000000,12.000000,45.000000,7.075000e+02,3.000000,6.000000,3.000000
50%,99.000000,67.000000,28.000000,63.000000,4.277000e+03,8.000000,16.000000,8.000000
75%,111.000000,86.000000,75.000000,78.000000,2.498800e+04,23.000000,44.000000,24.000000
max,266.000000,100.000000,574.000000,100.000000,3.579764e+07,69.000000,497.000000,303.000000


In [7]:
reviews.head()

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
0,m/0814255,Andrew L. Urban,False,Urban Cinefile,Fresh,NaN,2010-02-06,A fantasy adventure that fuses Greek mythology...
1,m/0814255,Louise Keller,False,Urban Cinefile,Fresh,NaN,2010-02-06,"Uma Thurman as Medusa, the gorgon with a coiff..."
2,m/0814255,NaN,False,FILMINK (Australia),Fresh,NaN,2010-02-09,With a top-notch cast and dazzling special eff...
3,m/0814255,Ben McEachen,False,Sunday Mail (Australia),Fresh,3.5/5,2010-02-09,Whether audiences will get behind The Lightnin...
4,m/0814255,Ethan Alter,True,Hollywood Reporter,Rotten,NaN,2010-02-10,What's really lacking in The Lightning Thief i...


In [8]:
reviews.describe()

,rotten_tomatoes_link,critic_name,top_critic,publisher_name,review_type,review_score,review_date,review_content
count,1130017,1111488,1130017,1130017,1130017,824081,1130017,1064211
unique,17712,11108,2,2230,2,814,8015,949181
top,m/star_wars_the_rise_of_skywalker,Emanuel Levy,False,New York Times,Fresh,3/5,2000-01-01,Parental Content Review
freq,992,8173,841481,13293,720210,90273,48019,267


In [9]:
reviews.review_content.iloc[0]

'A fantasy adventure that fuses Greek mythology to contemporary American places and values. Anyone around 15 (give or take a couple of years) will thrill to the visual spectacle'

In [10]:
movies_selected_columns = ['rotten_tomatoes_link', 'movie_title', 'critics_consensus']
movies_selected = movies[movies_selected_columns]
movies_selected.dropna(inplace=True)
print(f"Number of movies with critics consensus: {movies_selected.shape[0]}")

Number of movies with critics consensus: 9134


/var/folders/4f/h14zcr_15ss0fltppyfwj1q80000gn/T/ipykernel_10155/3745288495.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_selected.dropna(inplace=True)


In [11]:
# Convert the 'rotten_tomatoes_link' column to object type
movies_selected['rotten_tomatoes_link'] = movies_selected['rotten_tomatoes_link'].astype(str)
reviews['rotten_tomatoes_link'] = reviews['rotten_tomatoes_link'].astype(str)
reviews.dropna(subset=['review_content'], inplace=True)
grouped_reviews = reviews.groupby('rotten_tomatoes_link')['review_content'].apply(list).reset_index()

/var/folders/4f/h14zcr_15ss0fltppyfwj1q80000gn/T/ipykernel_10155/1682456736.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_selected['rotten_tomatoes_link'] = movies_selected['rotten_tomatoes_link'].astype(str)


In [12]:
full_reviews = grouped_reviews.merge(movies_selected, on='rotten_tomatoes_link', how='inner')
# Drop where review_content is missing
full_reviews

,rotten_tomatoes_link,review_content,movie_title,critics_consensus
0,m/0814255,[A fantasy adventure that fuses Greek mytholog...,Percy Jackson & the Olympians: The Lightning T...,Though it may seem like just another Harry Pot...
1,m/0878835,"[Like Holofcener's previous pictures, Please G...",Please Give,Nicole Holofcener's newest might seem slight i...
2,m/10,[10 (1979) is known for its numerical rating s...,10,Blake Edwards' bawdy comedy may not score a pe...
3,m/1000013-12_angry_men,"[A film with texture, humour and relevance at ...",12 Angry Men (Twelve Angry Men),Sidney Lumet's feature debut is a superbly wri...
4,m/1000079-20000_leagues_under_the_sea,[[The] embodiment of Disney at his best -- fam...,"20,000 Leagues Under The Sea","One of Disney's finest live-action adventures,..."
...,...,...,...,...
9127,m/zoolander,[Making a movie that's stupid fun must be real...,Zoolander,"A wacky satire on the fashion industry, Zoolan..."
9128,m/zoolander_2,"[Like its predecessor, the best bits of this s...",Zoolander 2,Zoolander No. 2 has more celebrity cameos than...
9129,m/zoom_2006,[The whole thing is a PG-rated greatest hits c...,Zoom,Lacking the punch and good cheer of The Incred...
9130,m/zootopia,[The variety of cute - and occasionally slight...,Zootopia,The brilliantly well-rounded Zootopia offers a...


In [13]:
print("Average number of characters in critics consensus:", full_reviews.critics_consensus.apply(lambda x: len(x) if type(x) == str else 0).mean())
print("Average number of words in critics consensus:", full_reviews.critics_consensus.apply(lambda x: len(x.split()) if type(x) == str else 0).mean())
print("Average number of sentences in critics consensus:", full_reviews.critics_consensus.apply(lambda x: len(x.split('.')) if type(x) == str else 0).mean())

Average number of characters in critics consensus: 143.11793692509855
Average number of words in critics consensus: 22.452474813841437
Average number of sentences in critics consensus: 2.0589137100306614


At this point, we have a dataframe with the movie title, critics consensus, and a list of reviews for each movie.

# Approaches

## Zero-shot

### Gemini

In [14]:
# Load environment variables from .env file
load_dotenv()

# Get API key from environment variable
api_key = os.getenv("GEMINI_API_KEY")
client = genai.Client(api_key=api_key)

In [15]:
# Function to generate a summary using the Gemini API
def generate_review_summary(reviews, movie_title, max_output_tokens=250, model="gemini-2.0-flash-lite"):
    # Join all reviews into a single string
    reviews_text = " ".join(reviews)
    
    prompt = f"""Summarize the following movie reviews for "{movie_title}". The summary should be around 2 sentences and around 25 words.
    
    Reviews:
    {reviews_text}
    
    Summary:"""
    
    try:
        response = client.models.generate_content(model="gemini-2.0-flash-lite", contents=prompt)
        return response.text
    except Exception as e:
        print(f"Error generating summary for {movie_title}: {e}")
        return None

In [16]:
# Create a sample of movies to test the approach (to save API costs)
sample_size = 5  # Adjust based on your needs
movie_sample = full_reviews.sample(sample_size, random_state=42)

In [17]:
# Generate summaries for the sample
summaries = []
for _, row in tqdm(movie_sample.iterrows(), total=len(movie_sample)):
    movie_title = row['movie_title']
    reviews_list = row['review_content']
    
    # Generate summary
    summary = generate_review_summary(reviews_list, movie_title)
    
    # Add to results
    summaries.append({
        'movie_title': movie_title,
        'critics_consensus': row['critics_consensus'],
        'generated_summary': summary,
        'num_reviews': len(reviews_list)
    })
    
    # Sleep to avoid rate limiting
    time.sleep(1)

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:10<00:00,  2.18s/it]


In [18]:
pd.set_option('display.max_colwidth', None)

# Create a DataFrame with the results
summaries_df = pd.DataFrame(summaries)

# Display the results
summaries_df[['movie_title', 'critics_consensus', 'generated_summary']]

,movie_title,critics_consensus,generated_summary
0,Hour of the Wolf (Vargtimmen),"Ingmar Bergman makes a successful foray into horror with Hour of the Wolf, infusing the demons that spring from creativity with his trademark psychological curiosity.","""Hour of the Wolf"" is a haunting and outlandish gothic fantasy, exploring existential dread and the impossible nature of relationships. It's a psychologically terrifying and visually striking film, though some consider it a less successful Bergman work.\n"
1,Girl Most Likely,"Largely witless and disappointingly dull, Girl Most Likely strands the gifted Kristen Wiig in a blandly hollow foray into scattershot sitcom territory.","""Girl Most Likely"" is a disappointing comedy, with many critics finding it to be unfunny and clichéd despite a talented cast, especially Kristen Wiig. The film's weak script, uneven tone, and sitcom-like qualities ultimately fail to deliver on its potential.\n"
2,Jurassic Park III,"Jurassic Park III is darker and faster than its predecessors, but that doesn't quite compensate for the franchise's continuing creative decline.","""Jurassic Park III"" is a formulaic but entertaining thrill ride, praised for its impressive dinosaur effects and fast-paced action. However, many reviewers found the plot and characters lacking, calling it a B-movie lacking the wonder of the original.\n"
3,Moms' Night Out,"Cheap-looking, unfunny, and kind of sexist to boot, Moms' Night Out is a disappointment from start to finish.","""Moms' Night Out"" is a faith-based comedy with mixed reviews; some find it funny and heartwarming while others see it as tedious and unfunny with dated themes. The film's portrayal of mothers and gender roles is also criticized.\n"
4,Doubt,"Doubt succeeds on the strength of its top-notch cast, who successfully guide the film through the occasional narrative lull.","""Doubt"" is a well-acted and thought-provoking adaptation of a stage play, featuring powerful performances by Meryl Streep and others. However, some critics feel the film is too theatrical and lacks cinematic impact.\n"


In [19]:
pd.reset_option('display.max_colwidth')

### OpenAI

*TO IMPLEMENT*

In [ ]:
prompt = f"""

"""

In [ ]:
m

## One-shot

*TO IMPLEMENT*

In [ ]:
##

## Few-shot

*TO IMPLEMENT*

## Fine-tuning

*TO IMPLEMENT*

# Evaluation

## ROUGE

*TO IMPLEMENT*

## BERTScore

Future work could include
1. Parallelizing the evaluation process to speed it up

https://huggingface.co/spaces/evaluate-metric/bertscore

*ADD EXPLANATION*

In [ ]:
# Function to calculate BERTScore
def calculate_bert_score(references, candidates):
    # Calculate precision, recall, and F1 scores
    P, R, F1 = score(candidates, references, lang="en", verbose=True)
    
    # Convert PyTorch tensors to Python floats
    P = [p.item() for p in P]
    R = [r.item() for r in R]
    F1 = [f.item() for f in F1]
    
    return P, R, F1

# Extract critics consensus and generated summaries
references = summaries_df['critics_consensus'].tolist()
candidates = summaries_df['generated_summary'].tolist()

# Calculate BERTScore
precision, recall, f1 = calculate_bert_score(references, candidates)

# Add scores to the DataFrame
summaries_df['bertscore_precision'] = precision
summaries_df['bertscore_recall'] = recall
summaries_df['bertscore_f1'] = f1

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:02<00:00,  2.36s/it]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 451.68it/s]

done in 2.37 seconds, 2.11 sentences/sec


,movie_title,critics_consensus,generated_summary,bertscore_f1
0,Hour of the Wolf (Vargtimmen),Ingmar Bergman makes a successful foray into h...,"""Hour of the Wolf"" is a haunting and outlandis...",0.873442
1,Girl Most Likely,"Largely witless and disappointingly dull, Girl...","""Girl Most Likely"" is a disappointing comedy, ...",0.875090
2,Jurassic Park III,Jurassic Park III is darker and faster than it...,"""Jurassic Park III"" is a formulaic but enterta...",0.876495
3,Moms' Night Out,"Cheap-looking, unfunny, and kind of sexist to ...","""Moms' Night Out"" is a faith-based comedy with...",0.880312
4,Doubt,Doubt succeeds on the strength of its top-notc...,"""Doubt"" is a well-acted and thought-provoking ...",0.873475


In [22]:
pd.set_option('display.max_colwidth', None)
summaries_df

,movie_title,critics_consensus,generated_summary,num_reviews,bertscore_precision,bertscore_recall,bertscore_f1
0,Hour of the Wolf (Vargtimmen),"Ingmar Bergman makes a successful foray into horror with Hour of the Wolf, infusing the demons that spring from creativity with his trademark psychological curiosity.","""Hour of the Wolf"" is a haunting and outlandish gothic fantasy, exploring existential dread and the impossible nature of relationships. It's a psychologically terrifying and visually striking film, though some consider it a less successful Bergman work.\n",19,0.872642,0.874244,0.873442
1,Girl Most Likely,"Largely witless and disappointingly dull, Girl Most Likely strands the gifted Kristen Wiig in a blandly hollow foray into scattershot sitcom territory.","""Girl Most Likely"" is a disappointing comedy, with many critics finding it to be unfunny and clichéd despite a talented cast, especially Kristen Wiig. The film's weak script, uneven tone, and sitcom-like qualities ultimately fail to deliver on its potential.\n",94,0.876370,0.873814,0.875090
2,Jurassic Park III,"Jurassic Park III is darker and faster than its predecessors, but that doesn't quite compensate for the franchise's continuing creative decline.","""Jurassic Park III"" is a formulaic but entertaining thrill ride, praised for its impressive dinosaur effects and fast-paced action. However, many reviewers found the plot and characters lacking, calling it a B-movie lacking the wonder of the original.\n",183,0.868532,0.884605,0.876495
3,Moms' Night Out,"Cheap-looking, unfunny, and kind of sexist to boot, Moms' Night Out is a disappointment from start to finish.","""Moms' Night Out"" is a faith-based comedy with mixed reviews; some find it funny and heartwarming while others see it as tedious and unfunny with dated themes. The film's portrayal of mothers and gender roles is also criticized.\n",44,0.876842,0.883810,0.880312
4,Doubt,"Doubt succeeds on the strength of its top-notch cast, who successfully guide the film through the occasional narrative lull.","""Doubt"" is a well-acted and thought-provoking adaptation of a stage play, featuring powerful performances by Meryl Streep and others. However, some critics feel the film is too theatrical and lacks cinematic impact.\n",216,0.862567,0.884662,0.873475


In [23]:
pd.reset_option('display.max_colwidth')

## LLM Evaluation

Ask an LLM to compare the generated summary with the critics consensus and rate the summary on a scale of 1-10.

Future work could include 
1. Prompt engineering to improve the quality of the evaluation. For example, we could specify specific dimensions/characteristics of the summary that we want to evaluate.
2. Using a different LLM for evaluation than we did for summary generation
3. Parallelizing the evaluation process to speed it up

In [24]:
def evaluate_summary_with_llm(reference, candidate, movie_title):
    """Simple LLM evaluation of summary quality"""
    prompt = f"""Rate how well this generated summary captures the critics consensus for the movie "{movie_title}".
    
    Critics Consensus: "{reference}"
    Generated Summary: "{candidate}"
    
    Rate on a scale of 1-10. Provide only the number, no explanation or other text.
    """
    
    try:
        response = client.models.generate_content(model="gemini-2.0-flash-lite", contents=prompt)
        return response.text.strip()
    except Exception as e:
        return f"Error: {str(e)}"

In [25]:
## LLM Evaluation

# Evaluate each summary
summaries_df['llm_score'] = None
for i, row in tqdm(summaries_df.iterrows(), total=len(summaries_df)):
    score = evaluate_summary_with_llm(
        row['critics_consensus'], 
        row['generated_summary'], 
        row['movie_title']
    )
    
    # Try to convert to float if possible
    try:
        summaries_df.at[i, 'llm_score'] = float(score)
    except (ValueError, TypeError):
        summaries_df.at[i, 'llm_score'] = None
    
    # Sleep to avoid rate limiting
    time.sleep(1)

100%|██████████| 5/5 [00:07<00:00,  1.52s/it]


In [26]:
# Display results
pd.set_option('display.max_colwidth', None)
summaries_df[['movie_title', 'critics_consensus', 'generated_summary', 'llm_score']]

,movie_title,critics_consensus,generated_summary,llm_score
0,Hour of the Wolf (Vargtimmen),"Ingmar Bergman makes a successful foray into horror with Hour of the Wolf, infusing the demons that spring from creativity with his trademark psychological curiosity.","""Hour of the Wolf"" is a haunting and outlandish gothic fantasy, exploring existential dread and the impossible nature of relationships. It's a psychologically terrifying and visually striking film, though some consider it a less successful Bergman work.\n",7.0
1,Girl Most Likely,"Largely witless and disappointingly dull, Girl Most Likely strands the gifted Kristen Wiig in a blandly hollow foray into scattershot sitcom territory.","""Girl Most Likely"" is a disappointing comedy, with many critics finding it to be unfunny and clichéd despite a talented cast, especially Kristen Wiig. The film's weak script, uneven tone, and sitcom-like qualities ultimately fail to deliver on its potential.\n",9.0
2,Jurassic Park III,"Jurassic Park III is darker and faster than its predecessors, but that doesn't quite compensate for the franchise's continuing creative decline.","""Jurassic Park III"" is a formulaic but entertaining thrill ride, praised for its impressive dinosaur effects and fast-paced action. However, many reviewers found the plot and characters lacking, calling it a B-movie lacking the wonder of the original.\n",6.0
3,Moms' Night Out,"Cheap-looking, unfunny, and kind of sexist to boot, Moms' Night Out is a disappointment from start to finish.","""Moms' Night Out"" is a faith-based comedy with mixed reviews; some find it funny and heartwarming while others see it as tedious and unfunny with dated themes. The film's portrayal of mothers and gender roles is also criticized.\n",4.0
4,Doubt,"Doubt succeeds on the strength of its top-notch cast, who successfully guide the film through the occasional narrative lull.","""Doubt"" is a well-acted and thought-provoking adaptation of a stage play, featuring powerful performances by Meryl Streep and others. However, some critics feel the film is too theatrical and lacks cinematic impact.\n",6.0


In [27]:
pd.reset_option('display.max_colwidth')

## Report Generation

*TO IMPLEMENT*